In [1]:
#import libraries
import numpy as np
import pandas as pd
import requests
import time

In [2]:
with open('C:/api_key/googleapi_key.txt','r') as f:
    line=f.read()

api_key=line.split('=')

In [3]:
google_map_api = api_key[1] #Used Given Api Key

#Place Ids For Different Places In Bangladesh
place_id=['ChIJ9wK5hdnAVTcRSUOp2Ljc6HQ','ChIJ-8P_PLO4VTcRrxLTX0dvgko','ChIJ4R3qFQzHVTcREbOP6IFjhZc',
          'ChIJnzJw0tNUUDcRgnP2MTT5jvU','ChIJFVwp_DZVUDcRkHoMen-KurE',
         'ChIJ09-VQKbYrDAR2QVpy1vMFVA','ChIJLUqJNZ3YrDAR26hzw4DENNY',
         'ChIJMdA4aqnv-zkREPTWDpU6-RA','ChIJWUvQti3w-zkRXOyjHkHrWKo',
         'ChIJmwGm_OYt4zkRyBj4h-aWpJ8','ChIJ-zMcpJUt4zkR9DwEcCz2VrY',
         'ChIJLxVHy3GQ_zkRUolxkCIhS_A','ChIJj9JQDEmF_zkRmga3k3tvrq0',
         'ChIJh-Ts-wc0VTcRSkHUmZWbBl0']

#took Radius(1000) and Type(Resturat)
rad=1000
keyword='restaurant'


lat=[]
long=[]

#For All Place Ids Extracted The Lat and Long
for i in range(0,len(place_id)):
    url=requests.get(f'https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id[i]}&key={google_map_api}')
    json=url.json()
    la=json['result']['geometry']['location']['lat']
    lat.append(la)
    ln=json['result']['geometry']['location']['lng']
    long.append(ln)

In [4]:
reslist=[]
#For Ever Lat and Long Found Nearby 60 Resturants then appended the Results in reslist
for i in range(0,len(lat)):
    url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={lat[i]},{long[i]}&radius={rad}&type={keyword}&key={google_map_api}'
    response= requests.get(url)
    json=response.json()


    
    reslist.append(json['results'])

    next_page = json['next_page_token']
    while next_page:
        time.sleep(2)
        url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?pagetoken={next_page}&key={google_map_api}'
        response= requests.get(url)
        json=response.json()
        reslist.append(json['results'])

        if 'next_page_token' not in json:
            break
        else:

            next_page = json['next_page_token']
    

In [5]:
len(reslist)

42

In [6]:
reslist[30]

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 25.7169539, 'lng': 89.26293009999999},
   'viewport': {'northeast': {'lat': 25.7182743802915,
     'lng': 89.2642693802915},
    'southwest': {'lat': 25.7155764197085, 'lng': 89.26157141970849}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
  'icon_background_color': '#FF9E67',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
  'name': 'New Sagorika Hotel And Restaurant',
  'opening_hours': {'open_now': False},
  'photos': [{'height': 4160,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/114103504542137351917">Sonjoy Sarker Roni</a>'],
    'photo_reference': 'Aap_uEBi2PBXhGvWJ1OvhWwwXNRhd_lKjLTymbIGv7u9oLe-Ibx5j3iEaJUuDqO4-3Kvjg4tV4br2qPw-2CW6k2PvEUiaeiJdwZmOEyuBWCDtMPnP9poTWA6i2MASCTigrcfQ2zT9bZtv9dK_Amz9R56h9Hl5YMZ-FC8ho_lzaBGL4oMbblr',
    'width': 3120}],
  'place_id': 'ChIJhXS5Bpkt4zkRUBYozCL

In [7]:
final_data=pd.DataFrame()
names=[]
lats=[]
longs=[]
ratings=[]
ppl_review=[]
affluences=[]

#For Every Result in reslist extracted name,latitude,longitude,rating,number of reviews,affluence
#Where the value was empty just appended with nan 
#Then finally Created a Dataframe with all values
for i in reslist:
    for j in i:
        name=j['name']
        names.append(name)
        
        lat = j['geometry']['location']['lat']
        lats.append(lat)
        
        lng = j['geometry']['location']['lng']
        longs.append(lng)
        if 'rating' in j:
            rating = j['rating']
            ratings.append(rating)
        else:
            rating=np.NaN
            ratings.append(rating)
        
        if 'user_ratings_total' in j:
            ppl_reviewd = j['user_ratings_total']
            ppl_review.append(ppl_reviewd)
            
        else:
            ppl_reviewd = np.NaN
            ppl_review.append(ppl_reviewd)
            
        if 'price_level' in j:
            affluence = j['price_level']
            affluences.append(affluence)
            
        else:
            affluence = np.NaN
            affluences.append(affluence)
        
        


cols={'name':names,'latitude':lats,'longitude':longs,'rating':ratings,'number of reviews':ppl_review,'affluence':affluences}
df = pd.DataFrame(cols)

final_data = final_data.append(df,ignore_index=True)


In [8]:
final_data

,name,latitude,longitude,rating,number of reviews,affluence
0,amontron,23.807410,90.368412,5.0,3.0,NaN
1,Bay Leaf Restaurant,23.809889,90.367557,3.7,950.0,2.0
2,VIP's Restaurant & Party Center,23.802421,90.353288,3.7,426.0,2.0
3,Kathbadam Restaurant,23.805770,90.365317,3.7,861.0,2.0
4,Green Mango Restaurant,23.805184,90.361153,3.8,220.0,2.0
...,...,...,...,...,...,...
835,Hungry Bird,22.701182,90.360448,4.5,6.0,NaN
836,Riyaz Tea Corner,22.694489,90.354007,4.3,8.0,NaN
837,Kabir Nasta Ghor,22.707421,90.351830,5.0,1.0,NaN
838,সৌরভ হোটেল এন্ড রেস্টুরেন্ট,22.707811,90.353486,NaN,NaN,NaN


In [9]:
len(final_data['name'].unique())

799

In [10]:
final_data

,name,latitude,longitude,rating,number of reviews,affluence
0,amontron,23.807410,90.368412,5.0,3.0,NaN
1,Bay Leaf Restaurant,23.809889,90.367557,3.7,950.0,2.0
2,VIP's Restaurant & Party Center,23.802421,90.353288,3.7,426.0,2.0
3,Kathbadam Restaurant,23.805770,90.365317,3.7,861.0,2.0
4,Green Mango Restaurant,23.805184,90.361153,3.8,220.0,2.0
...,...,...,...,...,...,...
835,Hungry Bird,22.701182,90.360448,4.5,6.0,NaN
836,Riyaz Tea Corner,22.694489,90.354007,4.3,8.0,NaN
837,Kabir Nasta Ghor,22.707421,90.351830,5.0,1.0,NaN
838,সৌরভ হোটেল এন্ড রেস্টুরেন্ট,22.707811,90.353486,NaN,NaN,NaN


In [11]:
#dropped the duplicate values
final_data=final_data.drop_duplicates()

In [21]:
#In affleunce column replaced the values 1.0,2.0,3.0,4.0 with $,$$,$$$,$$$$ respectively
final_data['affluence']=np.where(final_data['affluence']=='1.0','$',final_data['affluence'])


<ipython-input-21-a2e8f560b3e7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['affluence']=np.where(final_data['affluence']=='1.0','$',final_data['affluence'])


In [13]:
final_data['affluence']=np.where(final_data['affluence']=='2.0','$$',final_data['affluence'])


<ipython-input-13-416c7933a464>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['affluence']=np.where(final_data['affluence']=='2.0','$$',final_data['affluence'])


In [14]:
final_data['affluence']=np.where(final_data['affluence']=='3.0','$$$',final_data['affluence'])


<ipython-input-14-b25b37eccb56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['affluence']=np.where(final_data['affluence']=='3.0','$$$',final_data['affluence'])


In [15]:
final_data['affluence']=np.where(final_data['affluence']=='4.0','$$$$',final_data['affluence'])

<ipython-input-15-75179468a6c7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['affluence']=np.where(final_data['affluence']=='4.0','$$$$',final_data['affluence'])


In [16]:
final_data['affluence'] = np.where(final_data['affluence']=='nan',np.NaN,final_data['affluence'])

<ipython-input-16-bc56ddb64c59>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['affluence'] = np.where(final_data['affluence']=='nan',np.NaN,final_data['affluence'])


In [36]:
final_data[final_data['affluence']=='$']


,name,latitude,longitude,rating,number of reviews,affluence
36,Sonia Restaurant,23.792154,90.373585,3.6,56.0,$
117,Yumm TatsteTic,23.751666,90.386592,4.3,108.0,$
261,Jomidaar Restaurant,24.899572,91.852196,3.9,231.0,$
377,Sadia's Kitchen,22.360989,91.821406,3.8,1714.0,$
393,Desh Kitchen,22.361588,91.813744,4.1,34.0,$
441,Bindu Hotel And Restaurant,24.374020,88.603169,3.8,689.0,$
452,Arrow Spoon,24.372970,88.602013,3.7,164.0,$
492,Sagor Canteen,24.369171,88.638451,3.8,94.0,$
493,Monir Tea Stall,24.368923,88.636149,4.3,12.0,$
495,Selim Food Corner,24.369944,88.637569,4.5,12.0,$


In [31]:
#created a csv file
final_data.to_csv(r'C:/assignment/{}.csv'.format(keyword))